In [127]:
%pip install --upgrade stanza[transformers]



In [131]:
import stanza

print(stanza.__file__)

/usr/local/lib/python3.10/dist-packages/stanza/__init__.py


In [132]:
pipe = stanza.Pipeline("en", processors="tokenize,coref")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                 |
---------------------------------------
| tokenize  | combined                |
| mwt       | combined                |
| coref     | ontonotes_electra-large |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: coref
INFO:stanza:Done loading processors!


In [140]:
import pandas as pd
file_path = 'NEU_TEST.csv'
df = pd.read_csv(file_path)
df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,neu-0,They originally trained as an occupational the...,their,436,Holly Kenyon,336,True,Jane Alison,410,False,http://en.wikipedia.org/wiki/Holly_Kenyon
1,neu-1,"Since May 2011 , they have written a biweekly ...",They,291,Postrel,206,True,Sally Satel,277,False,http://en.wikipedia.org/wiki/Virginia_Postrel
2,neu-2,They advocated treating offenders like the men...,their,352,Karl Menninger,366,True,Thomas Szasz,399,False,http://en.wikipedia.org/wiki/Karl_Menninger
3,neu-3,"At age 17 , Rinker moved to Seattle and worked...",their,280,Rinker,215,False,Bailey,318,True,http://en.wikipedia.org/wiki/Mildred_Bailey
4,neu-4,"They wintered in 1836 - 1837 at New Orleans , ...",they,340,Stewart,273,True,Alfred Jacob Miller,313,False,http://en.wikipedia.org/wiki/William_Drummond_...


In [138]:
import json

def flatten(internal_list):
    flattened_list = []
    for item in internal_list:
        if isinstance(item, list):
            flattened_list.extend(item)
        else:
            flattened_list.append(item)
    return tuple(flattened_list)

def get_antecedents(index):
  offset_pron = df.loc[index, 'Pronoun-offset']
  A_coref = df.loc[index, 'A-coref']
  B_coref = df.loc[index, 'B-coref']

  true_antecedent = None
  false_antecedent_A = None
  false_antecedent_B = None


  if A_coref == True:
    true_antecedent = df.loc[index, 'A']
  elif B_coref == True:
    true_antecedent = df.loc[index, 'B']
  else:
    true_antecedent = None

  if A_coref == False:
    false_antecedent_A = df.loc[index, 'A']

  if B_coref == False:
    false_antecedent_B = df.loc[index, 'B']

  false_antecedents = [antecedent for antecedent in [false_antecedent_A, false_antecedent_B] if antecedent is not None]

  gold_cluster = (true_antecedent, offset_pron)
  false = (false_antecedents, offset_pron)

  false_cluster = flatten(false)

  return gold_cluster, false_cluster


def calc_TP_FN_FP(pron_offset, gold_ant, false_ant, document):

  TP = 0
  FP = 0
  FN = 0

  ant_list = []

  for s in document.sentences:
    for w in s.words:
      d = w.to_dict()
      # if d['start_char'] == pron_offset:
      start_char = d.get('start_char')
      if start_char is not None and start_char == pron_offset:
        corf = d.get('coref_chains', [])
        # ant_list = []
        for c in corf:
          # print(dir(c))
          jsn = c.to_json()
          # print(jsn)
          antecedent = jsn.get('representative_text')
          ant_list.append(antecedent)

  # print(ant_list)
  if any(gold_ant in string for string in ant_list):
    # print('yes')
    TP += 1
  else:
    FN += 1

  if any(false_ant in string for string in ant_list):
    # print('yes')
    FP += 1

  # to fix issues of "Melinda Medich , his girlfriend and a minor at the time of Byrne's death"
  if TP > 0 and FP > 0:
    FP -= 1


  return TP, FP, FN



def calc_prec_recall(TP, FN, FP):

  precision = TP / (TP + FP)
  recall = TP / (TP + FN)

  f1_score = 2 * (precision * recall) / (precision + recall)

  return precision, recall, f1_score




In [139]:
true_pos = []
false_neg = []
false_pos = []

error_count = 0


for index, row in df.iterrows():

  text = df.loc[index, 'Text']

  try:
    document = pipe(text)
  except IndexError:
    error_count+=1


  pron_ind = df.loc[index, 'Pronoun-offset']
  gold, false = get_antecedents(index)

  gold_ant = str(gold[0])
  false_ant = str(false[0])
  pron_offset = int(pron_ind)

  TP, FP, FN = calc_TP_FN_FP(pron_offset, gold_ant, false_ant, document)

  true_pos.append(TP)
  false_neg.append(FN)
  false_pos.append(FP)

total_TP = sum(true_pos)
total_FN = sum(false_neg)
total_FP = sum(false_pos)

IndexError: list index out of range

In [126]:
prec, recall, f1 = calc_prec_recall(total_TP, total_FN, total_FP)
print(f'precision: {prec}')
print(f'recall: {recall}')
print(f'f1: {f1}')

precision: 1.0
recall: 0.8
f1: 0.888888888888889
